## Joining on Non-Equi Operators

In [1]:
import pandas as pd
import janitor
import numpy as np 

/home/sam/pyjanitor/janitor/functions/then.py:14: UserWarning: registration of accessor <class 'pandas_flavor.register.register_dataframe_method.<locals>.inner.<locals>.AccessorMethod'> under name 'then' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  def then(df: pd.DataFrame, func: Callable) -> pd.DataFrame:


In [2]:
from pathlib import Path
filename = Path('.').cwd().parents[2]/'flights.csv'
df = pd.read_csv(filename, sep = '\t', names=['orig','dest','orig_time', 'dest_time'], parse_dates  = ['orig_time', 'dest_time'])
df = df.factorize_columns(['orig','dest']).iloc[:, 2:]
df.columns = [ent.split("_")[0] if ent.endswith('enc') else ent for ent in df]
df.columns = ['takeoff','landing','orig','dest']
df = df.assign(start=df.landing+pd.Timedelta(minutes=45), end=df.landing+pd.Timedelta(hours=3))
# indexes = [ 0, 8, 52, 60, 424, 118162, 117029, 120018, 119639]
# df = df.loc[indexes]
df

,takeoff,landing,orig,dest,start,end
0,2021-11-27 07:15:00,2021-11-27 08:55:00,0,0,2021-11-27 09:40:00,2021-11-27 11:55:00
1,2021-11-27 20:05:00,2021-11-28 00:50:00,1,1,2021-11-28 01:35:00,2021-11-28 03:50:00
2,2021-11-27 21:00:00,2021-11-27 21:35:00,2,2,2021-11-27 22:20:00,2021-11-28 00:35:00
3,2021-11-27 21:15:00,2021-11-27 22:25:00,0,2,2021-11-27 23:10:00,2021-11-28 01:25:00
4,2021-11-26 11:40:00,2021-11-26 14:45:00,3,3,2021-11-26 15:30:00,2021-11-26 17:45:00
...,...,...,...,...,...,...
127421,2022-01-29 13:45:00,2022-01-29 15:35:00,18,27,2022-01-29 16:20:00,2022-01-29 18:35:00
127422,2022-01-29 11:20:00,2022-01-29 12:20:00,27,2,2022-01-29 13:05:00,2022-01-29 15:20:00
127423,2022-01-29 16:00:00,2022-01-29 20:15:00,94,143,2022-01-29 21:00:00,2022-01-29 23:15:00
127424,2022-01-29 09:20:00,2022-01-29 10:55:00,27,27,2022-01-29 11:40:00,2022-01-29 13:55:00


In [3]:
# df.merge(df, left_on='dest', right_on='orig').loc[lambda f: f.takeoff_y.between(f.start_x, f.end_x) & (f.orig_x != f.orig_y)]


In [4]:
#%timeit df.merge(df, left_on='dest', right_on='orig').loc[lambda f: f.takeoff_y.between(f.start_x, f.end_x) & (f.orig_x != f.orig_y)]

In [5]:
#%%timeit
out = (df
 .conditional_join(
     df, 
     ('end', 'takeoff' ,'>='), 
   # ('start', 'takeoff', '<='), 
     ('orig','orig','!='), 
     ('dest', 'orig', '=='), 
     use_numba=True)
)
out


(array([     0,      1,      2, ..., 127423, 127424, 127425]),
 array([     9,   5474,   7647, ..., 115642,  36169, 102005]))

In [6]:

# %%timeit
outer = (df
 .conditional_join(
     df, 
   #  ('end', 'takeoff' ,'>='), 
     ('start', 'takeoff', '<='), 
     ('orig','orig','!='), 
    ('dest', 'orig', '=='), 
     use_numba=False)
)
outer

(array([   424,    424,    424, ..., 122523, 122523, 124752]),
 array([     3,     11,     14, ..., 115768, 125899, 125899]))

In [7]:
out.sort_values(out.columns.tolist(), ignore_index=True).equals(outer.sort_values(outer.columns.tolist(), ignore_index=True))

AttributeError: 'tuple' object has no attribute 'sort_values'

In [ ]:
#https://stackoverflow.com/q/61948103/7175713 
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 
                    'value_1': [2,5,7,1,3,4]})

df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 
                    'value_2A': [0,3,7,12,0,2,3,1], 
                    'value_2B': [1,5,9,15,1,4,6,3]})

# df2 = df2.sample(len(df2))

# df1 = pd.concat([df1]*20)
# df2 = pd.concat([df2]*20)

In [ ]:
df1.conditional_join(df2, ('id','id','=='), ('value_1','value_2A', '>'), ('value_1', 'value_2B', '<'),use_numba=True)

left         right                  
    id value_1    id value_2A value_2B
0    2       3     2        2        4

In [ ]:
df1.conditional_join(df2, ('id','id','=='), ('value_1','value_2A', '>'), ('value_1', 'value_2B', '<'),use_numba=False)

left         right                  
    id value_1    id value_2A value_2B
0    2       3     2        2        4

In [ ]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
5,3,4


In [ ]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,5
2,1,7,9
3,1,12,15
4,2,0,1
5,2,2,4
6,2,3,6
7,3,1,3


Join on equi and non-equi operators is possible:

In [ ]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       5     1        3        5
1    1       7     1        7        9
2    2       1     2        0        1
3    2       3     2        2        4
4    2       3     2        3        6

The default join is inner. left and right joins are supported as well:

In [ ]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='left',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       2   NaN      NaN      NaN
1    1       5   1.0      3.0      5.0
2    1       7   1.0      7.0      9.0
3    2       1   2.0      0.0      1.0
4    2       3   2.0      2.0      4.0
5    2       3   2.0      3.0      6.0
6    3       4   NaN      NaN      NaN

In [ ]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='right',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0  NaN     NaN     1        0        1
1  1.0     5.0     1        3        5
2  1.0     7.0     1        7        9
3  NaN     NaN     1       12       15
4  2.0     1.0     2        0        1
5  2.0     3.0     2        2        4
6  2.0     3.0     2        3        6
7  NaN     NaN     3        1        3

Join on just the non-equi joins is also possible:

In [ ]:
df1.conditional_join(
        df2,
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
        how='inner',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       2     3        1        3
1    1       5     2        3        6
2    2       3     2        2        4
3    3       4     1        3        5
4    3       4     2        3        6

Join on not equal -> !=

In [ ]:
df1.conditional_join(
        df2,
        ('id', 'id', "!=")
    )

left         right                  
     id value_1    id value_2A value_2B
0     1       2     2        0        1
1     1       2     2        2        4
2     1       2     2        3        6
3     1       2     3        1        3
4     1       5     2        0        1
5     1       5     2        2        4
6     1       5     2        3        6
7     1       5     3        1        3
8     1       7     2        0        1
9     1       7     2        2        4
10    1       7     2        3        6
11    1       7     3        1        3
12    2       1     3        1        3
13    2       3     3        1        3
14    2       1     1        0        1
15    2       1     1        3        5
16    2       1     1        7        9
17    2       1     1       12       15
18    2       3     1        0        1
19    2       3     1        3        5
20    2       3     1        7        9
21    2       3     1       12       15
22    3       4     1        0        1
23    3       4     1        3        5
24    3       4     1        7        9
25    3       4     1       12       15
26    3       4     2        0        1
27    3       4     2        2        4
28    3       4     2        3        6

If the columns from both dataframes have nothing in common, a single indexed column is returned:

In [ ]:
(df1.select_columns('value_1')
    .conditional_join(
        df2.select_columns('val*'),
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
    )
)

,value_1,value_2A,value_2B
0,2,1,3
1,5,3,6
2,3,2,4
3,4,3,5
4,4,3,6


Selection of relevant columns within `conditional_join`: 

In [ ]:
df1.conditional_join(
        df2,
        ('id', 'id', "<"),
        df_columns = 'id',
        right_columns = 'id'
    )

,left,right
,id,id
0,1,2
1,1,2
2,1,2
3,1,3
4,1,2
5,1,2
6,1,2
7,1,3
8,1,2


Column renaming is also possible:

In [ ]:
df1.conditional_join(
        df2,
        ('id', 'id', "<"),
        df_columns = {'id':'df_id'},
        right_columns = {'id':'right_id'}
    )

,df_id,right_id
0,1,2
1,1,2
2,1,2
3,1,3
4,1,2
5,1,2
6,1,2
7,1,3
8,1,2
9,1,2
